<a href="https://colab.research.google.com/github/szhang004/Sarcasm_Detection/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About**: The dataset is a collection of news headlines from TheOnion and HuffPost.

**Content**: Each record consists of three attributes: 
is_sarcastic: 1 if the record is sarcastic, 0 otherwise
headline: the headline of the news article
article_link: link to the original news article

**Task**: Train and evaluate a machine learning/deep learning algorithm to classify between news articles with and without sarcasm. We recommend either using recurrent models or pretrained Transformer architectures to complete this task (check out the Lesson 6: Fine-Tune BERT notebook for some tips!). 

In [5]:
import torch

if torch.cuda.is_available():        
    device = torch.device("cuda")

In [6]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CAIS++/Sarcasm Detection/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/CAIS++/Sarcasm Detection


In [7]:
import pandas as pd

df = pd.read_json("/content/drive/MyDrive/CAIS++/Sarcasm Detection/archive/Sarcasm_Headlines_Dataset.json", lines = True)

df.sample(10)

,article_link,headline,is_sarcastic
14125,https://local.theonion.com/vegetarian-begins-s...,"vegetarian begins sad, private routine of scan...",1
5009,https://entertainment.theonion.com/actress-lea...,actress leaves porn past behind with new cinem...,1
21887,https://www.huffingtonpost.com/entry/verizon-u...,verizon and unions reach 'tentative agreement'...,0
19253,https://www.huffingtonpost.com/entry/stop-talk...,"stop talking about 'screen time,' start thinki...",0
4648,https://www.huffingtonpost.com/entry/sarah-pal...,try your hand at making sarah palin's donald t...,0
11839,https://www.theonion.com/visa-fires-bob-dole-1...,visa fires bob dole,1
9828,https://www.huffingtonpost.com/entry/kerry-was...,kerry washington just summed up what we're all...,0
25828,https://entertainment.theonion.com/fred-willar...,fred willard a huge hit at counseling session,1
7389,https://local.theonion.com/60-year-old-hippie-...,60-year-old hippie pitied by 40-year-old punk,1
12774,https://www.huffingtonpost.com/entry/what-it-t...,what it takes to land a book deal,0


In [8]:
headlines = df.headline.values
labels = df.is_sarcastic.values

print(headlines)
print(labels)
type(headlines)
type(labels)

["former versace store clerk sues over secret 'black code' for minority shoppers"
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse"
 "mom starting to fear son's web series closest thing she will have to grandchild"
 ... 'reparations and obama'
 'israeli ban targeting boycott supporters raises alarm abroad'
 'gourmet gifts for the foodie 2014']
[0 0 1 ... 0 0 0]


numpy.ndarray

In [ ]:
!pip install textattack


In [ ]:
train_aug = headlines.copy()
from textattack.augmentation import EmbeddingAugmenter
aug = EmbeddingAugmenter()

augmented_labels = pd.DataFrame()
augmented_headlines = pd.DataFrame()

for i, headline in enumerate(headlines):
  aug.augment(headline)
  labels = np.insert(labels, 2*i, labels[2*i+1])

print(headlines)
print(labels)

In [9]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
max_len = 0

# For every sentence...
for article in headlines:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(article, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max headline length: ', max_len)

Max headline length:  66


In [12]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

augments = []

# For every sentence...
for article in headlines:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        article,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 100,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', headlines[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  former versace store clerk sues over secret 'black code' for minority shoppers
Token IDs: tensor([  101,  2280, 18601,  3401,  3573,  7805,  9790,  2015,  2058,  3595,
         1005,  2304,  3642,  1005,  2005,  7162,  4497,  7347,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [13]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Calculate the number of samples to include in each set.
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

22,702 training samples
4,007 validation samples


In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [15]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import torch.nn as nn

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    hidden_dropout_prob = .23,
    attention_probs_dropout_prob = .23
)



# class BertClassifier(nn.Module):
#     def __init__(self):
#         super(BertClassifier, self).__init__()
#         self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

#         # we want our output as probability so, in the evaluation mode, we'll pass the logits to a softmax layer
#         self.dropout = nn.Dropout(0.1) 

#         self.softmax = torch.nn.Softmax(dim = 1) # last dimension

#     def forward(self, x, attn_mask = None, labels = None):

#         outputs = self.bert(input_ids=input_ids,
#                              attention_mask=attention_mask)
#         sequence_output = self.dropout(outputs[0])

#       # last_hidden_state_cls = outputs[0][:, 0, :]      X
#       # logits = self.classifier(last_hidden_state_cls)  X

#         logits = self.classifier(sequence_output[:,0,:])
 
#         return logits

# model = BertClassifier()

model.cuda()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.23, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.23, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [16]:
# Define the optimizer
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon
                )


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [18]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 0 the gradients
        model.zero_grad() 

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.       
        result = model(b_input_ids, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    710.    Elapsed: 0:00:21.
  Batch    80  of    710.    Elapsed: 0:00:40.
  Batch   120  of    710.    Elapsed: 0:01:00.
  Batch   160  of    710.    Elapsed: 0:01:20.
  Batch   200  of    710.    Elapsed: 0:01:40.
  Batch   240  of    710.    Elapsed: 0:02:01.
  Batch   280  of    710.    Elapsed: 0:02:21.
  Batch   320  of    710.    Elapsed: 0:02:41.
  Batch   360  of    710.    Elapsed: 0:03:01.
  Batch   400  of    710.    Elapsed: 0:03:21.
  Batch   440  of    710.    Elapsed: 0:03:41.
  Batch   480  of    710.    Elapsed: 0:04:01.
  Batch   520  of    710.    Elapsed: 0:04:22.
  Batch   560  of    710.    Elapsed: 0:04:42.
  Batch   600  of    710.    Elapsed: 0:05:02.
  Batch   640  of    710.    Elapsed: 0:05:22.
  Batch   680  of    710.    Elapsed: 0:05:42.

  Average training loss: 0.30
  Training epcoh took: 0:05:57

Running Validation...
  Accuracy: 0.92
  Validation Loss: 0.21
  Validation took: 0:00:22

====

In [21]:
import pandas as pd

eval_df = pd.read_json("/content/drive/MyDrive/CAIS++/Sarcasm Detection/archive/Sarcasm_Headlines_Dataset_v2.json", lines = True)

half_df = len(eval_df) // 10
eval_df = eval_df.iloc[:half_df,]


eval_df.sample(10)

eval_headlines = eval_df.headline.values
eval_labels = eval_df.is_sarcastic.values


In [22]:
max_len = 0

# For every sentence...
for article in eval_headlines:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    eval_input_ids = tokenizer.encode(article, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max headline length: ', max_len)

Max headline length:  26709


In [23]:
# Tokenize all of the sentences and map the tokens to their word IDs.
eval_input_ids = []
eval_attention_masks = []


# For every sentence...
for article in eval_headlines:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    eval_encoded_dict = tokenizer.encode_plus(
                        article,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 100,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    eval_input_ids.append(eval_encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    eval_attention_masks.append(eval_encoded_dict['attention_mask'])

# Convert the lists into tensors.
eval_input_ids = torch.cat(eval_input_ids, dim=0)
eval_attention_masks = torch.cat(eval_attention_masks, dim=0)
eval_labels = torch.tensor(eval_labels)

eval_dataset = TensorDataset(eval_input_ids, eval_attention_masks, eval_labels)

eval_dataloader = DataLoader(
            eval_dataset,  # The training samples.
            sampler = RandomSampler(eval_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [24]:

import pandas as pd
import numpy as np

print('Predicting labels for {:,} test sentences...'.format(len(eval_input_ids)))

# Put model in evaluation mode
model.eval()

predictions , true_labels = [], []


# iterate over test data
for batch in validation_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)


Predicting labels for 2,861 test sentences...


In [25]:
#print(predictions[1])
for i, prediction in enumerate(predictions):
  predictions[i] = np.argmax(prediction, axis = 1)



# print(true_labels)
print("\n")
# print(predictions)

In [26]:

# type(predictions)
# type(true_labels)

# predictions = np.array(predictions)
# true_labels = np.array(predictions)

# print(predictions)

flat_predictions = []
flat_true_labels = []

for batch in predictions:
  for prediction in batch:
    flat_predictions.append(prediction)

for batch in true_labels:
  for label in batch:
    flat_true_labels.append(label)


print(flat_predictions)
print("\n")
print(flat_true_labels)

[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [27]:

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np


# Build confusion matrix
cf_matrix = confusion_matrix(flat_true_labels, flat_predictions)
print(cf_matrix)

from sklearn.metrics import classification_report
print(classification_report(flat_true_labels, flat_predictions))

[[2152   94]
 [ 220 1541]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      2246
           1       0.94      0.88      0.91      1761

    accuracy                           0.92      4007
   macro avg       0.92      0.92      0.92      4007
weighted avg       0.92      0.92      0.92      4007

